In [2]:
from field import F

# Generator
# G = F(85408008396924667383611388730472331217)
P = 17
G = F(3, P)

for i in range(0, P):
    print(i, G ** i)

G / 2

0 1
1 3
2 9
3 10
4 13
5 5
6 15
7 11
8 16
9 14
10 8
11 7
12 4
13 12
14 2
15 6
16 1


10

In [6]:
from polynomial import Polynomial
from field import F

P = 17
f = lambda x: F(x, P)
# f = lambda x: x
p = Polynomial([-1, 0, 8], f)
q = Polynomial([1,  1], f)

(p + q)([1, 2, 3])

[9, 0, 7]

In [6]:
import merkle

ls = ["A", "B", "C", "D", "E"]
hs = [merkle.hash_leaf(l) for l in ls]

for h in hs:
    print(h)

root = merkle.commit(hs)
print("root", root)

i = 2
proof = merkle.open(hs, i)
merkle.verify(proof, root, hs[i], i)

559aead08264d5795d3909718cdd05abd49572e84fe55590eef31a88a08fdffd
df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c
6b23c0d5f35d1b11f9b683f0b0a617355deb11277d91ae091d399c655b87940d
3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0ebfda1a01ec05745d43
a9f51566bd6705f7ea6ad54bb9deb449f795582d6529a0e22207b8981233ec58
root 2db1790243fe117685d21ed0ff5005d9832e5f32bf5b2b02cddf0f07a34421b2


True

In [3]:
import polynomial

p = polynomial.interp([1, 2], [1, 2])
p.degree()

1

In [2]:
# FRI domain
from field import find_generator, check_generator, get_primitive_root
from utils import is_prime

# Find L
n = 16
p = 337

assert is_prime(p), f'{p} is not prime'

# Find generator
g = find_generator(p)
assert g != None, "Generator not found"
print(f'generator g = {g}')

# Check order of the group generated by g
check_generator(g, p)

# Primitive Nth root
w = get_primitive_root(g, p, n)

# Evaluation domain L
L = [pow(w, i, p) for i in range(0, n)]
print(f'L = {L}')

generator g = 10
L = [1, 191, 85, 59, 148, 297, 111, 307, 336, 146, 252, 278, 189, 40, 226, 30]


In [2]:
import numpy as np
import random
import fri
from field import F
import polynomial
from polynomial import Polynomial
import merkle
import iop
from utils import is_pow2, is_prime

### Setup ###
# FRI domain L
# message (M) -> poly degree <= M - 1 -> RS code (N) <= |L| size of evaluation domain
# N = |L| = 2**K
# L = [1, w, w^2, ..., w^(N - 1)], w is Nth root of unity
# prime field F_p, |F_p| > |L|, N divides p - 1 (needed for finding primitive root of unity)

# Lenght of message
M = 8
ys = [random.randint(0, 1) for _ in range(M)]
print("ys:", ys)

# P = 1 + 407 * (1 << 119)
# Prime
P = 337
# Length of RS code
N = 16
# Nth primitive root
w = 191

# Expansion factor from message length M to RS code length N
# exp_factor * M = N
EXP_FACTOR = 2

# assert M < N < P
assert is_prime(P), f'{P} is not prime'
assert is_pow2(N), f'{N} is not a power of 2'

# TODO: plot
L = fri.domain(w, N, P)
p = polynomial.interp(L[:M], ys, lambda x: F(x, P))

### Commit ###
iop = iop.Writer()
prover = fri.Prover(
    N = N,
    P = P,
    w = w,
    poly = p,
    iop = iop,
    exp_factor = EXP_FACTOR
)

prover.commit()
print(prover.codewords)

### Query ###
verifier = fri.Verifier(
    N = N,
    P = P,
    w = w,
    challenges = iop.challenges,
    merkle_roots = iop.merkle_roots,
    exp_factor = EXP_FACTOR
)

# Random indexes without replacement
NUM_QUERIES = 2
assert NUM_QUERIES <= N

rands = [int(r) for r in np.random.choice(range(N), size=NUM_QUERIES, replace=False)]
for i in rands:
    (vals, proofs, codeword) = prover.prove(i)
    verifier.verify(i, vals, proofs, codeword)


ys: [1, 1, 1, 1, 1, 0, 1, 1]
[[1, 1, 1, 1, 1, 0, 1, 1, 81, 170, 9, 171, 246, 330, 334, 0], [135, 61, 40, 325, 317, 288, 230, 48], [19, 158, 173, 34], [28, 28]]


In [2]:
# Polynomials

# 0 <= P(x) <= 9 integer 
# for all 1 <= x <= 1,000,000

# C(x) = x*(x-1)*...*(x-9)
# C(x) = 0 if 0 <= x <= 9
def c(x):
    v = 1
    for i in range(10):
        v *= (x - i)
    return v

# Z(x) = (x-1)*(x-2)*...*(x-1e6)
def z(x, n):
    v = 1
    for i in range(1, n + 1):
        v *= (x - i)
    return v

# C(P(x)) = 0 for 1 <= x <= 1,000,000
# C(P(x)) = Z(x)D(x)

# Polynomial commitment
# 1. Merkle tree of P(x) and D(x) at 1,000,000,000 points
# 2. Verifier randomly selects 16 values between 1 and 1,000,000,000
#    and asks the prover to provide the Merkle branches of P(x) and D(x)
# 3. Verifier checks
#    - Merkle proofs
#    - C(P(x)) = Z(x)D(x), P(x) and D(x) are provided in the Merkle proof

# Bivariate polynomial
# f(x) = polynomial with degree < 1,000,000
# Find g(x,y) such that g(x, x^1000) = f(x)

# 1. Polynomial commintment on g(x, y)
# for {(x, y^1000) for 1 <= x <= N and 1 <= y <= N}
# N = 1,000,000,000

# 2. Verifier picks randomly picks a few rows and columns
#    and for each row and column, asks for a few sample of points
#    one point in the sample is on the diagnol (x, x^1000)

# 3. Prover replies with does points and Merkle proofs

# TODO: why use bivariate polynomials?
# 4. - Verifier checks Merkle proofs
#    - Check polynomial corresponds to a low degree polynomial.
#      Degree less than number of samples requested.

In [1]:
# Number Theoretic Transform (NTT) - FFT on modular arithmetic

# Recursive fft
def fft_rec(f: list[int], xs: list[int], p: int) -> list[int]:
    """
    f = polynomial of degree < N
    xs[i] = x^i, where x is a Nth primitive root of unity 
    p = x mod p
    Returns [f(xs[0]), f(xs[1]), ..., f(xs[N-1])]
    """
    n = len(f)
    assert n & (n - 1) == 0, f'{n} is not a power of 2'
    assert len(f) == len(xs)
    
    if n == 1:
        return f
    # Optional - check x is a Nth primitive root of unity
    x = xs[1]
    assert pow(x, n, p) == 1, f'{x}^{n} mod {p} != 1'
    assert pow(x, n // 2, p) == p - 1, f'{x}^({n} / 2) mod {p} != -1'

    f_even = fft_rec(f[::2], xs[::2], p)
    f_odd = fft_rec(f[1::2], xs[::2], p)
    ys = [0 for _ in xs]
    
    h = n // 2
    for i in range(h):
        # -1 = w^(n/2)
        # -w^i = -1 * w^i = w^(n/2 + i)
        # f(x)  = f_even(x^2) + x * f_odd(x^2)
        # f(-x) = f_even(x^2) - x * f_odd(x^2)
        # If f_even(w^(2i)) and f_odd(w^(2i)) are known,
        # then f(w^i) and f(-w^i) can be quickly calculated
        ys[i] = (f_even[i] + xs[i] * f_odd[i]) % p
        ys[h + i] = (f_even[i] - xs[i] * f_odd[i]) % p
    
    return ys

def inv_fft(primitive_root, xs):
    pass

def eval(f: list[int], xs: list[int], p: int) -> list[int]:
    ys = [0 for _ in range(xs)]
    for xi in xs:
        x = 1
        y = 0
        for c in f:
            y += c * x
            x *= xi
        y %= p
    
    return ys

P = 17
N = 16
w = 3

# f  = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
f = [i for i in range(N)]
ws = [pow(w, i, P) for i in range(P - 1)]
assert len(set(ws)) == P - 1

# print("xs", xs)

ys = [c for c in f]
# n = len(xs)
n = N

# FFT without recursion
# TODO - test
def fft(f: list[int], ws: list[int], p: int) -> list[int]:
    n = len(f)
    assert n & (n - 1) == 0, f'{n} is not a power of 2'
    # assert len(f) == len(ws)

    ys = [c for c in f]

    # Split evens and odds
    # TODO - check code, use bit reversal?
    evens = [0 for _ in range(n // 2)]
    odds = [0 for _ in range(n // 2)]
    k = n
    while k > 0:
        h = k // 2
        for i in range(0, n, k):
            for j in range(0, h):
                evens[j] = ys[i + 2 * j]
                odds[j] = ys[i + 2 * j + 1]
    
            for j in range(0, h):
                ys[i + j] = evens[j]
                ys[i + j + h] = odds[j]
    
        k //= 2

    # Debug
    # ys = [f'a{i}' for i in ys]
    # print("ys", ys)

    # Merge
    k = n
    s = 2
    while k > 1:
        for i in range(0, n, s):
            h = s // 2
            for j in range(i, i + h):
                f_even = ys[j]
                f_odd = ys[j + h]
                # wi = (w^j)^k = w^(j * k % n) at loop k, 
                #      so the wi at next loop = w^(j * (k // 2) % n)
                wi = ws[(j * (k // 2)) % n]

                ys[j] = (f_even + wi * f_odd) % p
                ys[j + h] = (f_even - wi * f_odd) % p

                # Debug
                # a = ((j * k) // 2) % n
                # print(k, i, j, a)
                # ys[j] = f'({f_even} + w^{a}{f_odd})'
                # ys[j + h] = f'({f_even} + w^{(n // 2 + a) % n}{f_odd})'

        # Debug
        # print(k, "----------------------")
        # for y in ys:
        #     print(y)

        s *= 2
        k //= 2

    return ys

N = 8

f = [i for i in range(N)]
ws = [pow(w, i, P) for i in range(P - 1)]
ys = fft(f, ws, P)

In [2]:
# [0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15]

def eval_poly(f: list[int], xs: list[int], p: int) -> list[int]:
    ys = [0 for _ in xs]
    for i, xi in enumerate(xs):
        x = 1
        y = 0
        for c in f:
            y += c * x
            x *= xi
        y %= p
        ys[i] = y
    
    return ys

P = 17
N = 8
w = 9

# f  = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
f = [i + 1 for i in range(N)]
ws = [pow(w, i, P) for i in range(N)]
# assert len(set(ws)) == P - 1

print("f", f)
print("ws", ws)
e = eval_poly(f, ws, P)
r = fft_rec(f, ws, P)
l = fft(f, ws, P)

print("e", e)
print("r", r)
print("l", l)
# assert r == e
# assert l == e

# print l

f [1, 2, 3, 4, 5, 6, 7, 8]
ws [1, 9, 13, 15, 16, 8, 4, 2]
e [2, 1, 12, 3, 13, 6, 14, 8]
r [2, 1, 12, 3, 13, 6, 14, 8]
l [2, 1, 12, 3, 13, 6, 14, 8]


In [8]:
# N-th root of unity
# Any number w such that w^N = 1 mod P

# Primitive N-th root of unity
# w^N = 1 and w^k != 1 for 0 < k < N 

# Steps to find the N-th primitive root of unity mod P
# 0. Pick a prime P -> This defines the finite field F[P]
#    TODO: Why prime?
#    - nonzero element has a multiplicative inverse
# 1. Consider the multiplicative group F[P]x = {1,2,3,...,P−1} 
# 2. Find a generator g of F[P]x
# 3. Find N = largest power of 2 dividing P−1
#    The subgroup of order N is:
#      H = { g^((P−1)/N)^i  for i = 0 ... N−1 }
# 4. To find a primitive N-th root of unity:
#    Let w = g^((P−1)/N) (g^(P-1) = 1 (Fermat's little theorem))
#    Verify:
#      - w^N = 1
#      TODO: why this check suffices
#      - w^(N/2) = -1

from field import find_generator
from utils import find_largest_pow_2

# 0. Pick a prime P
P = 2**251 + 17 * 2**192 + 1
P = 17

# 2. Find a generator g
g = find_generator(P)
print("g = ", g)

# 3. Find N = largest power of 2 dividing P - 1
k = find_largest_pow_2(P - 1)
N = 2**k
print("k =", k)
print("n =", N)

w = pow(g, ((P - 1) // N), P)

assert pow(w, N, P) == 1
assert pow(w, N // 2, P) == (-1 % P)

print(f"{N} th primitive root of unity =", w)

g =  3
k = 4
n = 16
16 th primitive root of unity = 3
